# Pygal

### Installations and Imports

In [1]:
# !pip install pygal
# !pip install cairosvg

In [2]:
import pandas as pd
import numpy as np

import pygal
from IPython.display import display, HTML

In [3]:
# Function for interactively loading Pygal graphs in Jupyter Notebook

html_pygal = u"""
    <!DOCTYPE html>
    <html>
        <head>
            <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
            <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js"></script>
        </head>
        <body><figure>{rendered_chart}</figure></body>
    </html>
"""
def pygal_plot(chart):
    rendered_chart = chart.render(is_unicode=True)
    plot_html = html_pygal.format(rendered_chart=rendered_chart)
    display(HTML(plot_html))

### Loading Data

In [4]:
class_data=pd.read_csv('https://raw.githubusercontent.com/jamwine/Learn-Pandas/master/Test.csv')
class_data.head()

,A,B,C,D,E,F,G,H,I,J,K,L,M
0,5,4,20,5,38,17,5,4,20,5,30,43,44
1,5,3,15,5,33,15,5,4,18,5,29,41,43
2,5,3,20,5,40,18,5,3,20,5,30,45,45
3,5,4,18,5,20,14,5,3,18,5,20,43,36
4,5,3,19,4,38,15,5,3,20,5,29,45,43


In [5]:
year_data=pd.read_csv('https://raw.githubusercontent.com/jamwine/Data/master/Class%20Result.csv')
bins=np.linspace(min(year_data["Total Marks"]),max(year_data["Total Marks"]),5)
year_data["Grade"]=pd.cut(year_data["Total Marks"],bins,include_lowest =True,labels=["Grade D","Grade C","Grade B","Grade A"])
year_data.head()

,Exam1,Exam2,Exam3,Total Marks,Percentage,Grade
0,240,232,232,704,93.87,Grade A
1,221,163,169,553,73.73,Grade C
2,244,236,239,719,95.87,Grade A
3,196,160,154,510,68.00,Grade C
4,234,196,180,610,81.33,Grade B


In [6]:
expenditure_df=pd.read_csv("https://raw.githubusercontent.com/jamwine/Data/master/Monthly%20Expenditure.csv")
expenditure_df.head()

,Date,Day,Category,Expenditure,Cost
0,03-06-2017,Saturday,Rent,Hostel,9500
1,03-06-2017,Saturday,Travel,Bus,212
2,03-06-2017,Saturday,Travel,Auto,50
3,03-06-2017,Saturday,Drinks,Tea,10
4,04-06-2017,Sunday,Drinks,Juice,10


In [7]:
stock_df=pd.read_csv('https://raw.githubusercontent.com/jamwine/Data/master/stock_data.csv')
stock_df.columns=["Date"]+list(stock_df.columns[1:])
stock_df['Date']=stock_df['Date'].apply(lambda x: pd.to_datetime(x,format='%d-%m-%Y').date())
stock_df.head()

,Date,Open,High,Low,Close,Volume,Symbol
0,2021-04-01,238.470,242.84,238.0501,242.35,30337982,MSFT
1,2021-03-31,232.910,239.10,232.3900,235.77,43623471,MSFT
2,2021-03-30,233.525,233.85,231.1000,231.85,24792012,MSFT
3,2021-03-29,236.590,236.80,231.8800,235.24,25227455,MSFT
4,2021-03-26,231.550,236.71,231.5500,236.48,25479853,MSFT


## Bar Graphs


In [8]:
grades_count = dict(year_data['Grade'].value_counts())

bar_chart = pygal.Bar(height=360, width=640, explicit_size=True, truncate_legend=-1) # truncate_legend=-1 fits the legends
bar_chart.title = "Grades Summary"
bar_chart.x_labels = grades_count.keys()
bar_chart.add('Students with Grade',grades_count.values())
bar_chart.x_title = "Final Grades"
bar_chart.y_title = "Number of Students"

pygal_plot(bar_chart)

* The SVG can be rendered in the browser directly using the `bar_chart.render_in_browser()` command.

* SVGs can be saved locally by `bar_chart.render_to_file('name.svg')`. 


In [9]:
hbar_chart = pygal.HorizontalBar(height=360, width=640, explicit_size=True)
hbar_chart.title = "Fetching maximum marks in each subject (A - M)"
hbar_chart.y_title = "Subject (A - M)"
hbar_chart.x_title = "Marks"
hbar_chart.x_labels = class_data.columns
hbar_chart.add('Maximum Marks', class_data.max())
pygal_plot(hbar_chart)

In [10]:
temp=(year_data[['Exam1','Exam2','Exam3']]>200).astype(int).sum(axis=1)
count=[(temp>i).value_counts()[True] for i in range(3)]

line_chart = pygal.Bar(height=360, width=640, explicit_size=True,truncate_legend=-1, rounded_bars=10, print_values=True)
line_chart.title = "Number of students scoring Marks > 200"
line_chart.y_title = "Number of students"  
line_chart.add('in atleast one exam', count[0], formatter = lambda x: "%s Students" %count[0])
line_chart.add('in atleast two exams', [{'value': count[1], 'xlink':'https://github.com/jamwine/Data/blob/master/Class%20Result.csv'}])
line_chart.add('in all exams', [{'value':count[2], 'style':'fill: pink; stroke:black; stroke-width:4'}])
pygal_plot(line_chart)

In [11]:
from pygal.style import Style
custom_style = Style(colors=('#E80080', '#404040', '#9BC850'))
bar_chart = pygal.Bar(height=360, width=640, explicit_size=True,truncate_legend=-1,style=custom_style)
bar_chart.title = "Examination Marks for first 5 students"
bar_chart.x_labels = map(lambda x: 'Id '+str(x), list(range(1,6)))
bar_chart.x_title = "Student ID"
bar_chart.y_title = "Marks"
bar_chart.add('First term Exam', year_data['Exam1'][:5])
bar_chart.add('Second term Exam', year_data['Exam2'][:5])
bar_chart.add('Third term Exam', year_data['Exam3'][:5])
pygal_plot(bar_chart)

In [12]:
bar_chart = pygal.StackedBar(height=360, width=640, explicit_size=True,truncate_legend=-1)
bar_chart.title = "Total Marks for first 10 students"
bar_chart.x_labels = map(lambda x: 'Student Id '+str(x), list(range(1,11)))
bar_chart.x_title = "Student ID"
bar_chart.y_title = "Total Marks"
bar_chart.add('First term Exam', year_data['Exam1'][:10])
bar_chart.add('Second term Exam', year_data['Exam2'][:10])
bar_chart.add('Third term Exam', year_data['Exam3'][:10])
pygal_plot(bar_chart)

## Pie Charts

In [13]:
expenses = dict(expenditure_df.groupby('Category')['Cost'].sum())

pie_chart = pygal.Pie(height=360, width=640, explicit_size=True)
pie_chart.title = 'Expense Summary'
for k,v in expenses.items():
    pie_chart.add(k, v)
pygal_plot(pie_chart)

In [14]:
from pygal.style import NeonStyle

pie_chart = pygal.Pie(height=360, width=640, explicit_size=True, half_pie=True, style=NeonStyle)
pie_chart.title = 'Expense Summary'
for k,v in expenses.items():
    pie_chart.add(k, v)
pygal_plot(pie_chart)

In [15]:
donut_chart = pygal.Pie(height=360, width=640, explicit_size=True, inner_radius=0.4,  print_values=True)
donut_chart.title = 'Grades Summary'
for k,v in grades_count.items():
    donut_chart.add(k, v)
pygal_plot(donut_chart)

In [16]:
from pygal.style import DarkStyle

donut_chart = pygal.Pie(height=360, width=640, explicit_size=True, inner_radius=0.4, half_pie=True, style=DarkStyle)
donut_chart.title = 'Grades Summary'
for k,v in grades_count.items():
    donut_chart.add(k, v)
pygal_plot(donut_chart)

In [17]:
temp=expenditure_df.groupby(['Day','Category'])['Cost'].sum().reset_index().set_index('Day')

pie_chart = pygal.Pie(height=360, width=640, explicit_size=True)
pie_chart.title = 'Daywise Total Expense for each category'
for day in temp.index.unique():
    pie_chart.add(day,[{'value':e[1], 'label':e[0]}  for e in (temp.loc[day].values)])
pygal_plot(pie_chart)

## Line Graphs

In [18]:
temp= stock_df[stock_df['Symbol']=='MSFT'].sort_values('Date')

line_chart = pygal.Line(height=360, width=640, explicit_size=True, x_label_rotation=40)
line_chart.title = 'MSFT Price Summary'
line_chart.y_title = 'Price'
line_chart.x_labels = temp['Date']
line_chart.add('Open', temp['Open'])
line_chart.add('Close', temp['Close'])
line_chart.add('High', temp['High'])
line_chart.add('Low', temp['Low'])

pygal_plot(line_chart)

In [19]:
temp=stock_df[['Date','Close','Symbol']].set_index('Date').sort_index()
close_df=pd.DataFrame()
for sym in stock_df['Symbol'].unique():
    close_df[sym]=temp[temp['Symbol']==sym]['Close']
close_df=close_df.diff().iloc[1:]
    
line_chart = pygal.HorizontalLine(height=360, width=640, explicit_size=True, x_value_formatter=lambda x:x.strftime("%d %b"))
line_chart.title = 'Daily difference in Closing Price'
line_chart.x_title = 'Price difference'
line_chart.x_labels = close_df.index
for col in close_df:
    line_chart.add(col,close_df[col])
pygal_plot(line_chart)

In [20]:
line_chart = pygal.StackedLine(height=360, width=640, explicit_size=True,fill=True, x_label_rotation=20)
line_chart.title = 'Total Marks in all exams'
line_chart.x_title = 'Student ID'
line_chart.y_title = 'Marks'
line_chart.x_labels = map(lambda x:'ID '+str(x), range(1, 16))
line_chart.add('Exam 1', year_data.iloc[:15]['Exam1'])
line_chart.add('Exam 2',  year_data.iloc[:15]['Exam2'])
line_chart.add('Exam 3',  year_data.iloc[:15]['Exam3'])
pygal_plot(line_chart)

## XY Plots

In [21]:
from pygal import Config
custom_config = Config(width = 640, height = 360, explicit_size = True)

In [22]:
sine_points = [(x, np.sin(np.radians(x))) for x in range(-180, 181, 10)]
cosine_points = [(x, np.cos(np.radians(x))) for x in range(-180, 181, 10)]

xy_chart = pygal.XY(custom_config)
xy_chart.title = 'sin(x) and cos(x) with strokes'
xy_chart.add('sin(x)', sine_points, show_dots = False)
xy_chart.add('cos(x)', cosine_points)
pygal_plot(xy_chart)

In [23]:
chart = pygal.XY(custom_config, stroke=False)
chart.x_title = 'values'
chart.title = 'SQUARES AND CUBES'
chart.x_labels = list(range(1,11))
chart.add('squares', [(x,x**2) for x in list(range(1,11))])
chart.add('cubes', [(x,x**3) for x in list(range(1,11))])
pygal_plot(chart)

In [24]:
exam_chart = pygal.XY(custom_config,stroke=False)

exam_chart.title = 'Exams Performance Comparision'
exam_chart.x_title = 'Exam1 Marks'
exam_chart.y_title = 'Exam2 & Exam3 Marks'

exam_chart.add('Exam 2', list(zip(year_data['Exam1'],year_data['Exam2'])))
exam_chart.add('Exam 3', list(zip(year_data['Exam1'],year_data['Exam3'])))

pygal_plot(exam_chart)

## Histogram Chart

In [25]:
def get_bins_bars(series, bins, getbins=False):
    """
    Function to get bins, bars and labels (required for histograms and pyramid charts.
    Returns series_bins, x_labels when parameter getbins=True, else returns bars
    """
    bars = []
    series_bins=pd.cut(series, bins, include_lowest = True).value_counts().sort_index()
    for index,value in zip(series_bins.index,series_bins.values):
        bars.append({'value':(value, round(index.left),round(index.right)),'label':str(round(index.left))+' to '+str(round(index.right))})
    if getbins:
        x_labels=[b.get('label') for b in bars]
        return series_bins, x_labels
    return bars

In [26]:
bars = get_bins_bars(series = year_data['Percentage'], bins=[40,60,75,90,95,100])

histogram_chart = pygal.Histogram(custom_config)
histogram_chart.add('Students', bars)
histogram_chart.title = "No. of Students by Percentage Group"
histogram_chart.x_title = "Percentage"
histogram_chart.y_title = "Number of Students"
pygal_plot(histogram_chart)

In [27]:
from pygal.style import LightGreenStyle

msft_bars = get_bins_bars(series = close_df['MSFT'], bins=[-10,-5,-2,0,2,5,10])
apple_bars = get_bins_bars(series = close_df['AAPL'], bins=[-10,-5,-2,0,2,5,10])

histogram_chart = pygal.Histogram(custom_config, legend_at_bottom=True, style= LightGreenStyle())
histogram_chart.title = 'Daily Price Changes of MSFT and AAPL'
histogram_chart.x_title = 'Price Range'
histogram_chart.y_title = 'Frequency'
histogram_chart.add('MSFT', msft_bars)
histogram_chart.add('AAPL', apple_bars)
pygal_plot(histogram_chart)

This graph is better visualized as a Pyramid chart.

## Pyramid Chart

In [28]:
msft_bins, x_labels = get_bins_bars(series = close_df['MSFT'], bins=[-10,-5,-2,0,2,5,10],getbins=True)
apple_bins, x_labels = get_bins_bars(series = close_df['AAPL'], bins=[-10,-5,-2,0,2,5,10],getbins=True)

pyramid_chart = pygal.Pyramid(custom_config, legend_at_bottom=True)
pyramid_chart.x_labels =  x_labels
pyramid_chart.y_title = 'Price Range'
pyramid_chart.x_title = 'Frequency'
pyramid_chart.title = 'Daily Price Changes of MSFT and AAPL'
pyramid_chart.add('MSFT', msft_bins)
pyramid_chart.add('AAPL', apple_bins)
pygal_plot(pyramid_chart)

## Box Plots

In [29]:
box_whisker_plot = pygal.Box(custom_config)
box_whisker_plot.title = 'Exam Statistics'

box_whisker_plot.add('Exam1', year_data['Exam1'])
box_whisker_plot.add('Exam2', year_data['Exam2'])
box_whisker_plot.add('Exam3', year_data['Exam3'])
pygal_plot(box_whisker_plot)

Use `box_mode="1.5IQR"` in the configuration to change whiskers length: Lower_whisker will be the first quartile minus 1.5 times the interquartile range and Upper_whisker will be the third quartile plus 1.5 times the interquartile range.

In [30]:
box_whisker_plot = pygal.Box(custom_config, box_mode="1.5IQR")
box_whisker_plot.title = 'Exam Statistics'

box_whisker_plot.add('Exam1', year_data['Exam1'])
box_whisker_plot.add('Exam2', year_data['Exam2'])
box_whisker_plot.add('Exam3', year_data['Exam3'])
pygal_plot(box_whisker_plot) 

**Tukey boxplot** is the default boxplot in most the visualization frameworks, it finds the outliers. Anything above the third quartile + 1.5 IQR for the upper boxplot is an outlier and anything below the first quartile - 1.5 IQR is an outlier.

In [31]:
box_whisker_plot = pygal.Box(custom_config, box_mode="tukey")
box_whisker_plot.title = 'Exam Statistics'

box_whisker_plot.add('Exam1', year_data['Exam1'])
box_whisker_plot.add('Exam2', year_data['Exam2'])
box_whisker_plot.add('Exam3', year_data['Exam3'])
pygal_plot(box_whisker_plot) 

In the Boxplot mode "Stdev" - Sample Standard Deviation:
* Upper Whisker = Highest point that's within the "Q2 + Sample Standard Deviation"
* Lower Whisker = Lowest data point that's greater than "Q2 -Sample Standard Deviation"

In [32]:
box_whisker_plot = pygal.Box(custom_config, box_mode="stdev")
box_whisker_plot.title = 'Exam Statistics'

box_whisker_plot.add('Exam1', year_data['Exam1'])
box_whisker_plot.add('Exam2', year_data['Exam2'])
box_whisker_plot.add('Exam3', year_data['Exam3'])
pygal_plot(box_whisker_plot) 

In the Boxplot mode "pstdev" - Population Standard Deviation:
* Upper Whisker = Highest point that's within the "Q2 + Population Standard Deviation"
* Lower Whisker = Lowest data point that's greater than "Q2 -Population Standard Deviation"

In [33]:
box_whisker_plot = pygal.Box(custom_config, box_mode="pstdev")
box_whisker_plot.title = 'Exam Statistics'

box_whisker_plot.add('Exam1', year_data['Exam1'])
box_whisker_plot.add('Exam2', year_data['Exam2'])
box_whisker_plot.add('Exam3', year_data['Exam3'])
pygal_plot(box_whisker_plot) 

## Advanced Charts

In [1]:
# dot_chart = pygal.Dot(x_label_rotation=30)
# # nameing a title
# dot_chart.title = 'A vs B vs C vs D'
# # namind different labels
# dot_chart.x_labels = ['Richards', 'DeltaBlue', 'Crypto', 'RayTrace', 'EarleyBoyer', 'RegExp', 'Splay', 'NavierStokes']
# # adding the data
# dot_chart.add('A', [6395, 2212, 7520, 7218, 12464, 1660, 2123, 8607])
# dot_chart.add('B', [8473, 9099, 1100, 2651, 6361, 1044, 3797, 9450])
# dot_chart.add('C', [3472, 2933, 4503, 5229, 5510, 1828, 9013, 4669])
# dot_chart.add('D', [43, 41, 59, 79, 144, 136, 34, 102])

In [35]:
# from pygal.maps.world import World

# worldmap_chart = pygal.maps.world.World()
# worldmap_chart.title = 'Some countries'
# worldmap_chart.add('F countries', ['fr', 'fi'])
# worldmap_chart.add('M countries', ['ma', 'mc', 'md', 'me', 'mg',
#                                    'mk', 'ml', 'mm', 'mn', 'mo',
#                                    'mr', 'mt', 'mu', 'mv', 'mw',
#                                    'mx', 'my', 'mz'])
# worldmap_chart.add('U countries', ['ua', 'ug', 'us', 'uy', 'uz'])
# pygal_plot(worldmap_chart)

In [36]:
# worldmap_chart = pygal.maps.world.World()
# worldmap_chart.title = 'Color mapping of temperature scalar field on March 20'
# worldmap_chart.add('below 0 Degrees', {
#     'ca': -5.35,'gl': -10,'mn':-0.7,'aq':-20,'ru': -5.1
# })
# worldmap_chart.add('Above 0 and below 15', {
#     'af': 12.6,'al': 11.4,'ad': 7.6,'tj':2,'ua':8.3,'ar': 14.8,'ee': 5.1,'cn': 6.95,'am': 7.15,'az': 11.95,'by': 6.15,'be': 9.55,'bt': 7.4,
#     'bo': 21.55,'at':6.35,'bg': 10.55,'ch':5.5,'cl':8.45,'cz':7.55,'de':8.55,'dk':7.5,'ee':5.1,'es':13.3,'fi':1.7,'fr':10.7,
#     'gb':8.45,'ge':5.8,'hr':10.4,'hu':9.75,'ie':9.3,'is':1.75,'it':13.45,'jp':11.15,'kg':1.75,'kr':5.7,'kp':11.5,'kz':6.4,
#     'li':5.65,'lt':6.2,'lu':8.65,'lv':5.6,'mc':13.55,'md':9.45,'me':10.55,'mk':9.7,'nl':9.25,'no':1.25,'np':8.25,'nz':10.55,
#     'pl':7.85,'pr':15,'rs':10.55,'ro':8.8,'tr':11.1,'se':2.1,'us':8.55,'si':8.1,'sk':6.7,'uz':12.05,'sm':11.85
# })
# worldmap_chart.add('Above 15 and below 25', {
#     'in': 23.5,'br': 24.95,'vn':24.45,'tw':20,'tz':22.23,'uy':17.55,'ug':22.8,'pt':15.15,'py':23.55,'rw':17.86,'bf': 20,
#     'co': 24.95,'eg': 22.1,'au': 21.65,'sa':24.85,'bd': 25,'bi':19.8,'bw':21.5,'cd':24.55,'cf':24.9,'cg':24,'ci':20,'cr':24.8,
#     'cv':23.4,'cy':18.45,'do':24.55,'dz':22.5,'tm':15.1,'tn':19.2,'ec':21.85,'gr':15.4,'gt':23.4,'hk':20,'hn':23.5,'ht':24.9,'id':23,
#     'il':19.4,'iq':21.4,'ir':17.25,'jm':24.95,'jo':18.3,'ke':24.75,'lb':16.4,'ly':21.8,'ma':17.1,'mg':22.65,'mu':22.35,'mt':19.2,
#     'mw':21.19,'mx':21,'mz':23.8,'ni':24.9,'pe':19.6,'pk':20.5,'ye':23.85,'zm':24.1,'zw':21,'st':23.75,'sy':17.75
# })
# worldmap_chart.add("Above 25", {
#     'bn': 26.85,'pg':25.5,'tg':27.15,'ph':25.85,'th':26.3,'tl':25.25,'ve':25.35,'sc':27.15,'pa':25.4,'my':25.4,'cm':26.6,
#     'cu': 25.2,'sn':27.85,'sd':26.9,'sr':25.7,'so':27.05,'ga': 25.1,'gw': 26.75,'bz': 25.3,'bj': 27.55,'ae': 27,'bh': 27.15,
#     'cm':26.6,'dj':28,'er':25.5,'gh':27,'gm':27.7,'gn':25.7,'gu':26,'gy':26,'kw':25.35,'lk':26.95,'ml':28,'mr':27.65,'mv':27.65,
#     'ne':27.15,'ng':26.8,'sg':26.45,'sl':26.05
# })
# pygal_plot(worldmap_chart)

In [37]:
# from pygal.style import RotateStyle
# rotate_style = RotateStyle('#75ff98')

# phases = ['Install', 
#           'Register', 
#           'Order', 
#           'Monthly Order', 
#           'Weekly Order', 
#           'Daily Order']

# funnel_chart = pygal.Funnel(width = 640,
#                             height = 360,
#                             explicit_size = True,
#                             style = rotate_style)

# funnel_chart.title = 'Food Delivery App Usage'
# funnel_chart.x_labels = phases

# funnel_chart.add('Install', 10064)
# funnel_chart.add('Register', 7473)
# funnel_chart.add('Order', 6395)
# funnel_chart.add('Monthly Order', 5254)
# funnel_chart.add('Weekly Order', 1805)
# funnel_chart.add('Daily Order', 429)

# pygal_plot(funnel_chart)

In [38]:
# from pygal.maps.world import World
# # creating object
# worldmap_chart = pygal.maps.world.World()
# # adding title
# worldmap_chart.title = 'Some countries'
# # add data
# worldmap_chart.add('F countries', ['fr', 'fi'])
# worldmap_chart.add('I countries', ['in', 'il','iq'])
# worldmap_chart.add('M countries', ['ma', 'mc', 'md', 'me', 'mg',
#                                    'mk', 'ml', 'mm', 'mn', 'mo',
#                                    'mr', 'mt', 'mu', 'mv', 'mw',
#                                    'mx', 'my', 'mz'])
# worldmap_chart.add('U countries', ['ua', 'ug', 'us', 'uy', 'uz'])

In [39]:
# worldmap_chart = pygal.maps.world.World()
# # adding the title.
# worldmap_chart.title = 'random data'
# # adding the random values.
# worldmap_chart.add('values', {
#   'af': 14,
#   'bd': 1,
#   'by': 3,
#   'cn': 1000,
#   'gm': 9,
#   'in': 1,
#   'ir': 314,
#   'iq': 129,
#   'jp': 7,
#   'kp': 6,
#   'pk': 1,
#   'ps': 6,
#   'sa': 79,
#   'so': 6,
#   'sd': 5,
#   'tw': 6,
#   'ae': 1,
#   'us': 43,
#   'ye': 28
# })

In [40]:
# worldmap_chart = pygal.maps.world.World()
# supra = pygal.maps.world.SupranationalWorld()
# # adding the values
# supra.add('Asia', [('asia', 1)])
# supra.add('Europe', [('europe', 2)])
# supra.add('Africa', [('africa', 3)])
# supra.add('North america', [('north_america', 4)])
# supra.add('South america', [('south_america', 5)])
# supra.add('Oceania', [('oceania', 6)])
# supra.add('Antartica', [('antartica', 7)])

In [41]:
# import requests
# url =  'https://api.github.com/search/repositories?q=language:python&sort=stars'
# r = requests.get(url)
# print("Status code:",r.status_code)

In [42]:
# response_dict = r.json()
# print(response_dict.keys())

In [43]:
# print("Total repositories:", response_dict['total_count'])

In [44]:
# # Explore information about the repositories.
# repo_dicts = response_dict['items']
# print("Repositories returned:",len(repo_dicts))

In [45]:
# repo_dict = repo_dicts[0]
# print("\nKeys:",len(repo_dict))
# for key in sorted(repo_dict.keys()):
#     print(key)

In [46]:
# print("\nSelected information about first repository:")
# print('Name:',repo_dict['name'])
# print('Owner:',repo_dict['owner']['login'])
# print('Stars:',repo_dict['stargazers_count'])
# print('Repository:',repo_dict['html_url'])
# print('Created:',repo_dict['created_at'])
# print('Updated:',repo_dict['updated_at'])
# print('Description:',repo_dict['description'])

In [47]:
# from pygal.style import LightColorizedStyle as LCS, LightenStyle as LS
# names, plot_dicts = [], []
# for repo_dict in repo_dicts:
#     names.append(repo_dict['name'])

#     # Get the project description, if one is available.
#     description = repo_dict['description']
#     if not description:
#         description = "No description provided."

#     plot_dict = {
#         'value': repo_dict['stargazers_count'],
#         'label': description,
#         'xlink': repo_dict['html_url'],
#         }
#     plot_dicts.append(plot_dict)

# # Make visualization.
# my_style = LS('#333366', base_style=LCS)
# my_style.title_font_size = 24
# my_style.label_font_size = 14
# my_style.major_label_font_size = 18

# my_config = pygal.Config()
# my_config.x_label_rotation = 45
# my_config.show_legend = False
# my_config.truncate_label = 15
# my_config.show_y_guides = False
# my_config.width = 1000

# chart = pygal.Bar(my_config, style=my_style)
# chart.title = 'Most-Starred Python Projects on GitHub'
# chart.x_labels = names

# chart.add('', plot_dicts)
# # chart.render_to_file('python_repos.svg')
# pygal_plot(chart)